In [1]:
import xarray as xr
from glob import glob
from os import path
import os
import numpy as np

# create NOAA OISST zarr data files



In [14]:
adir = 'F:/data/sat_data/sst/noaa_oisst/www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/'
dir_pattern_zarr = adir + 'avhrr_zarr/'
dir_pattern = adir + 'avhrr/'
pattern = dir_pattern + '/*/*.nc'
files = glob(pattern)
print('number of files:',len(files)

In [3]:
%%time
#open dataset, this will take a while
ds=xr.open_mfdataset(files,combine='nested',concat_dim='time',decode_cf=False,mask_and_scale=False)
ds.close()
ds_all = ds.isel(zlev=0)
ds_all

Wall time: 7min 31s


In [6]:
#remove any duplicates
_, index = np.unique(ds_all['time'], return_index=True)
ds_all=ds_all.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
ds_all = ds_all.chunk({'time':1000,'lat':300,'lon':300})
ds_all

<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 14032)
Coordinates:
  * time     (time) float64 1.339e+03 1.34e+03 1.341e+03 ... 1.537e+04 1.537e+04
    zlev     float32 0.0
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.375 359.625 359.875
Data variables:
    sst      (time, lat, lon) int16 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
    anom     (time, lat, lon) int16 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
    err      (time, lat, lon) int16 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
    ice      (time, lat, lon) int16 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.0
    title:          Daily-OI-V2, final, Data (Ship, Buoy, AVHRR, GSFC-ice)
    History:        Version 2.0
    creation_date:  2011-05-04
    Source:         NOAA/National Climatic Data Center
    Contact:        Dick Reynolds, email: Richard.W.Reynolds@noaa.gov & Chuny...

In [8]:
#output data to zarr format
ds_all.to_zarr(dir_pattern_zarr, consolidated=True)

In [9]:
ds = xr.open_zarr(dir_pattern_zarr, consolidated=True)
ds

<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 14032)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.375 359.625 359.875
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2020-01-31
    zlev     float32 ...
Data variables:
    anom     (time, lat, lon) float32 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
    err      (time, lat, lon) float32 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
    ice      (time, lat, lon) float32 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
    sst      (time, lat, lon) float32 dask.array<chunksize=(1000, 300, 300), meta=np.ndarray>
Attributes:
    Contact:        Dick Reynolds, email: Richard.W.Reynolds@noaa.gov & Chuny...
    Conventions:    CF-1.0
    History:        Version 2.0
    Source:         NOAA/National Climatic Data Center
    creation_date:  2011-05-04
    title:          Daily-OI-V2, final, Data (Ship, Buoy, AVHRR, GSFC-ice)